<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
# !pip install -U pip

In [2]:
# !pip install -U torch

In [3]:
!pip install scikit-learn==1.1.3 -q
!pip install matplotlib==3.5.2 -q
!pip install tqdm -q
!pip install catboost
!pip install torch==2.6.0 -q
!pip install --upgrade Pillow
!pip install --upgrade transformers

ERROR: Could not find a version that satisfies the requirement torch==2.6.0 (from versions: 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2)
ERROR: No matching distribution found for torch==2.6.0


In [24]:
# Все импорты тут:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoConfig
import torch
from tqdm import notebook

# Работа с пайплайнами
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# класс для работы с пропусками
from sklearn.impute import SimpleImputer

# Разделение данных на выборки
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

# Классы для преобразования данных
from sklearn.preprocessing import StandardScaler

# Модели
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

# Метрики
from sklearn.metrics import f1_score

# создание констант
RANDOM_STATE = 42
TEST_SIZE = 0.2

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Хорошее оформление импортов! \
Импорты собраны в одной ячейке, разделены на функциональные группы пустой строкой.</font>

In [5]:
# Открытие данных через конструкцию try-except
try:
    toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')

except:
    toxic_comments = pd.read_csv('datasets/toxic_comments.csv')

In [6]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [7]:
pd.set_option('display.max_colwidth', None)
toxic_comments.head()

,Unnamed: 0,text,toxic
0,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0


In [8]:
df_tweets = toxic_comments.sample(1000).reset_index(drop=True)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert", force_download=True)

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

In [10]:
tokenized = df_tweets['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512))

In [11]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

display(padded.shape, attention_mask.shape)

(1000, 512)

(1000, 512)

In [12]:
config = AutoConfig.from_pretrained("https://huggingface.co/unitary/toxic-bert/resolve/main/config.json", force_download=True)

/opt/anaconda3/envs/practicum/lib/python3.9/site-packages/transformers/utils/hub.py:610: FutureWarning: Using `from_pretrained` with the url of a file (here https://huggingface.co/unitary/toxic-bert/resolve/main/config.json) is deprecated and won't be possible anymore in v5 of Transformers. You should host your file on the Hub (hf.co) instead and use the repository ID. Note that this is not compatible with the caching system (your file will be downloaded at each execution) or multiple processes (each process will download the file in a different temporary file).
  warnings.warn(


config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

In [13]:
model = AutoModel.from_pretrained("unitary/toxic-bert", config=config) 

In [14]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

## Обучение

In [15]:
X = np.concatenate(embeddings)
y = df_tweets['toxic']

In [16]:
# разделим модель на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [21]:
# LogisticRegression
lr_model = LogisticRegression(solver='liblinear', random_state=RANDOM_STATE)
lr_model.fit(X_train, y_train)
lr_f1_val = cross_val_score(lr_model, X_train, y_train, cv=5, scoring='f1').mean()
print(f'Показатель F1 модели LogisticRegression: {lr_f1_val}')

Показатель F1 модели LogisticRegression: 0.9334889478337753


In [18]:
# CatBoostClassifier
catboost_model = CatBoostClassifier(random_seed=RANDOM_STATE, silent=True)
catboost_params = {'depth': range(5,8),
                   'learning_rate': [0.025,0.05]}

catboost_grid = GridSearchCV(
    catboost_model,
    param_grid=catboost_params, 
    verbose=5,
    cv=5,
    scoring='f1')


catboost_grid.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ......depth=5, learning_rate=0.025;, score=0.903 total time=  41.5s
[CV 2/5] END ......depth=5, learning_rate=0.025;, score=0.963 total time=  41.5s
[CV 3/5] END ......depth=5, learning_rate=0.025;, score=0.880 total time=  41.6s
[CV 4/5] END ......depth=5, learning_rate=0.025;, score=0.963 total time=  42.2s
[CV 5/5] END ......depth=5, learning_rate=0.025;, score=0.929 total time=  41.0s
[CV 1/5] END .......depth=5, learning_rate=0.05;, score=0.903 total time=  41.6s
[CV 2/5] END .......depth=5, learning_rate=0.05;, score=0.963 total time=  42.3s
[CV 3/5] END .......depth=5, learning_rate=0.05;, score=0.923 total time=  41.1s
[CV 4/5] END .......depth=5, learning_rate=0.05;, score=0.963 total time=  41.9s
[CV 5/5] END .......depth=5, learning_rate=0.05;, score=0.929 total time=  41.6s
[CV 1/5] END ......depth=6, learning_rate=0.025;, score=0.867 total time= 1.2min
[CV 2/5] END ......depth=6, learning_rate=0.025;,

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x13224de80>,
             param_grid={'depth': range(5, 8), 'learning_rate': [0.025, 0.05]},
             scoring='f1', verbose=5)

In [20]:
print(f'Параметры лучшей модели CatBoostRegression: {catboost_grid.best_params_}')
print(f'Показатель F1 модели CatBoostRegression: {catboost_grid.best_score_}')

Параметры лучшей модели CatBoostRegression: {'depth': 6, 'learning_rate': 0.025}
Показатель F1 модели CatBoostRegression: 0.943133903133903


Таким образом, лучшей оказалась модель CatBoostRegression. Тестирование необходимо провести над этой моделью.

In [22]:
cat_predictions = catboost_grid.best_estimator_.predict(X_test)

In [25]:
print(f'Показатель F1 лучшей модели на тестовой выборке: {f1_score(y_test, cat_predictions)}')

Показатель F1 лучшей модели на тестовой выборке: 0.9411764705882353


## Выводы

Таким образом, были обучены 2 модели, способные классифицировать комментарии на позитивные и токсичные. 
Лучшей моделью оказалась модель CatBoostRegression с показателем F1 на кросс-валидации: 0.943133903133903
Модель прошла тестирование с показателем F1, равным: 0.9411764705882353

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны